In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
sys.path.append('../crazyflie-audio/python/')
from bin_selection import select_frequencies

def buzzer_signal(source):
    from signals import generate_signal
    kwargs = dict(
        duration_sec=DURATION_SEC, 
        max_dB=-10,
        Fs=FS,
    )
    frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)
    if source == 'mono4125':
        return generate_signal(signal_type='mono', frequency_hz=4125, **kwargs)
    elif source == 'random':
        return generate_signal(signal_type='random', **kwargs)
    elif source == 'multi':
        phase = np.random.uniform(0, 2*np.pi)
        signal = generate_signal(signal_type='mono', frequency_hz=frequencies[1], **kwargs, phase_offset=phase)
        for f in frequencies[2:]:
            phase = np.random.uniform(0, 2*np.pi)
            signal += generate_signal(signal_type='mono', frequency_hz=f, **kwargs, phase_offset=phase)
        return signal
    elif source == 'sweep':
        min_freq = 100
        max_freq = 4400
        bins = select_frequencies(n_buffer=N_BUFFER, fs=FS, min_freq=min_freq, max_freq=max_freq)
        
        signal = np.zeros(int(FS * kwargs['duration_sec']))
        kwargs['duration_sec'] = 0.5
        
        idx = 0
        for f in frequencies[bins]:
            mono = generate_signal(signal_type='mono', frequency_hz=f, **kwargs)
            signal[idx:idx+len(mono)] = mono
            idx += len(mono)
        return signal
    else:
        raise ValueError(source)
    
signals = {}
for key in ['mono4125', 'random', 'multi', 'sweep']:
    try:
        signal = pd.read_pickle(f'results/{key}.pk')
        print(f'read {key}')
    except:
        signal = buzzer_signal(key)
        pd.to_pickle(signal, f'results/{key}.pk')
        print(f'saved {key}')
    signals[key] = signal

In [ ]:
def matrix_slices(df_matrix, frequencies, distances):
    from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
    import ipywidgets as widgets

    def update_figure(slice_f, slice_d):
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(10, 5)
        axs[0].semilogy(distances, df_matrix[slice_f, :])
        axs[0].set_title(f'frequency = {frequencies[slice_f]:.0f} Hz')
        axs[0].set_xlabel('distance [cm]')
        axs[0].set_ylim(1e-1, 10)
        #axs[0].set_ylim(np.min(df_matrix), np.max(df_matrix))

        axs[1].semilogy(frequencies, df_matrix[:, slice_d])
        axs[1].set_title(f'distance = {distances[slice_d]:.0f} cm')
        axs[1].set_xlabel('frequency [Hz]')
        axs[1].set_ylim(1e-1, 10)
        #axs[1].set_ylim(np.min(df_matrix), np.max(df_matrix))
        plt.show()

    interactive_plot = interact(update_figure, 
                                slice_f=IntSlider(200, min=0, max=len(frequencies)-1, continuous_update=False), 
                                slice_d=IntSlider(50, min=0, max=len(distances)-1, continuous_update=False))

In [ ]:
from simulation import generate_room, generate_rir

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
room = generate_room(distance_cm=50, yaw_deg=0, ax=ax, single_mic=True)

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(10, 5)
room = generate_room(distance_cm=0, yaw_deg=0, ax=axs[0])
axs[0].set_title(0)
room = generate_room(distance_cm=0, yaw_deg=90, ax=axs[1])
axs[1].set_title(90)
axs[0].legend(loc='lower left')
axs[1].legend(loc='lower left')

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS
frequencies_theo = np.fft.rfftfreq(N_BUFFER, 1/FS)

for distance_cm in [10, 50]:
    fig, axs = plt.subplots(1, 2)
    fig.suptitle(distance_cm)
    Hs = generate_rir(frequencies_theo, distance_cm=distance_cm, single_mic=True, ax=axs[0])
    for H in Hs:
        axs[1].plot(frequencies_theo, np.abs(H))

In [ ]:
from simulation import get_signals_f
room = generate_room(distance_cm=50, single_mic=True)

signals_f = get_signals_f(room, signals['sweep'])
#signals_f = get_signals_f(room, signals['random'])
#signals_f = get_signals_f(room, signals['multi'])
#signals_f = get_signals_f(room, signals['mono4125'])

frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)

fig, axs = plt.subplots(signals_f.shape[1], 1, sharex=True, squeeze=False)
fig.set_size_inches(8, 2.5 * signals_f.shape[1])
for i in range(signals_f.shape[1]):
    axs[i, 0].pcolormesh(range(signals_f.shape[0]), frequencies, np.log10(np.abs(signals_f[:, i].T)))
    axs[i, 0].set_ylabel("frequency [Hz]")
    axs[i, 0].set_title(f"mic{i}")
axs[i, 0].set_xlabel("time index")

In [ ]:
room.compute_rir()

n_times = 500
times = np.arange(n_times) / FS * 1e3
plt.plot(times, room.rir[0][0][:n_times])
plt.title('room impulse response')
plt.xlabel('time [ms]')
plt.show()

# Distance-frequency matrix

In [ ]:
from simulation import get_freq_slice_pyroom
import progressbar

distances = np.linspace(0, 60, 100)
frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)
df_matrix = np.zeros((len(frequencies), len(distances)))

mic_idx = 1

signal = signals['multi']
with progressbar.ProgressBar(max_value=len(distances)-1) as bar:
    for j, distance_cm in enumerate(distances):
        #room = generate_room(distance_cm=distance_cm, single_mic=True)
        #signals_f = get_signals_f(room, signal)
        #spec = np.sum(np.abs(signals_f), axis=1) / (signals_f.shape[1] * N_BUFFER) # times x frequencies
        #spec_time_avg = np.mean(spec, axis=0) 
        #df_matrix[:, j] = spec_time_avg
        slice_mics = get_freq_slice_pyroom(frequencies, distance_cm, signal=signal)
        #df_matrix[:, j] = np.mean(slice_mics, axis=0) 
        df_matrix[:, j] = slice_mics[mic_idx]
        bar.update(j)

In [ ]:
from simulation import get_freq_slice_theory
df_matrix_theo = np.empty((len(frequencies), len(distances)))

for j, distance_cm in enumerate(distances):
    slice_mics = get_freq_slice_theory(frequencies, distance_cm)
    df_matrix_theo[:, j] = slice_mics[mic_idx]
    #H = generate_rir(frequencies, distance_cm=distance_cm, single_mic=False)[mic_idx] # 
    #df_matrix_theo[:, j] = np.abs(H)

In [ ]:
min_freq = 100
max_freq = 20000

fig = plt.figure()
fig.set_size_inches(10, 5)
freq_start = int(min_freq / FS * N_BUFFER)
freq_end = int(max_freq / FS * N_BUFFER)
distance_end = 1000
plt.pcolormesh(distances[:distance_end], frequencies[freq_start:freq_end], np.log10(df_matrix[freq_start:freq_end, :distance_end]))
matrix_slices(df_matrix[freq_start:freq_end, :distance_end], frequencies[freq_start:freq_end], distances[:distance_end])

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 5)
plt.pcolormesh(distances, frequencies[freq_start:], np.log10(df_matrix_theo[freq_start:]))
matrix_slices(df_matrix_theo[freq_start:], frequencies[freq_start:], distances)

In [ ]:
from constants import SPEED_OF_SOUND

#frequencies_slices = [671, 3125, 4156, 8000]
#frequencies_slices = [1750, 2375, 3125, 3875]
frequencies_slices = [359,  703, 1406]

max_distance = 60
fig, axs = plt.subplots(len(frequencies_slices), sharey=True, sharex=True)
fig.suptitle('frequency slices')
fig.set_size_inches(10, 5)
for i, f in enumerate(frequencies_slices):
    idx = np.argmin(np.abs(frequencies-f))
    slice_ = df_matrix[idx, distances < max_distance]
    #slice_ -= np.mean(slice_)
    
    d_constr = SPEED_OF_SOUND / (2 * f) * 100
    axs[i].plot(distances[distances < max_distance], slice_, label=f'{f}Hz, period={d_constr:.0f}cm')
    axs[i].set_ylabel('response')
    axs[i].legend(loc='upper right')
    
    
    # peak from constr. interference is expected when the wavelength equals k * distance/2
    # wavelength_constr = c / f_constr = distance / 2
    num_peaks = (distances[-1]-distances[0])//d_constr + 1
    [axs[i].axvline(x=d, color='C1') for d in np.arange(num_peaks)*d_constr]
    
    fft = np.fft.rfft(slice_)[1:]
    df = distances[1] - distances[0]
    freq = np.fft.rfftfreq(len(slice_), df)[1:]
    plt.figure()
    plt.plot(1/freq, np.abs(fft))
    plt.axvline(d_constr, color='C1')
axs[-1].set_xlabel('distance [cm]')

In [ ]:
from simulation import get_setup, Y_OFFSET, D0
distance_slices = [5, 10, 20, 30, 40, 50]

max_frequency = 5000
min_frequency = 1000
freq_indices = np.where((frequencies < max_frequency) & (frequencies > min_frequency))[0]
sparse_indices = np.linspace(freq_indices[0], freq_indices[-1], 32).astype(int)


fig, axs = plt.subplots(len(distance_slices), sharey=True, sharex=True)
fig.suptitle('frequency slices')
fig.set_size_inches(10, 10)
for i, d in enumerate(distance_slices):
    idx = np.argmin(np.abs(distances - d))
    slice_ = df_matrix[freq_indices, idx]**2
    axs[i].plot(frequencies[freq_indices], slice_, label=f'{d}cm')
    axs[i].set_xlabel('frequency [Hz]')
    axs[i].set_ylabel('response')
    axs[i].legend(loc='upper right')
    
    slice_ = df_matrix[sparse_indices, idx]**2
    axs[i].scatter(frequencies[sparse_indices], slice_, label=f'{d}cm', color='C1')
    
    source, mic_positions = get_setup(distance_cm=d, single_mic=False)
    source_image = [source[0], -source[1]]
    mic = mic_positions[mic_idx] 
    
    # peak from constr. interference is expected when the wavelength equals k * distance/2
    # wavelength_constr = c / f_constr = distance
    #d1 = 2 * np.sqrt((d * 1e-2 + Y_OFFSET)**2 + D0**2) # m
    #delta = d1 - D0
    delta = np.linalg.norm(source_image - mic) - np.linalg.norm(source - mic)
    
    f_constr = SPEED_OF_SOUND / delta  # Hz
    [axs[i].axvline(x=f+1000, color='C1') for f in np.arange(2)*f_constr]
    
    fft = np.fft.rfft(slice_)[1:]
    df = frequencies[sparse_indices[1]] - frequencies[sparse_indices[0]]
    freq = np.fft.rfftfreq(len(slice_), df)[1:]
    plt.figure()
    plt.plot(1/freq, np.abs(fft))
    plt.axvline(f_constr, color='C1')

# Experiment twins

In [ ]:
from wall_analysis import get_psd
import progressbar 

df_total = pd.DataFrame(columns=['signals_f', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 'frequencies'])

exp_name = '2020_11_23_wall2'; 
n_times = 300

min_freq=100
max_freq=4400
bins = select_frequencies(n_buffer=N_BUFFER, fs=FS, min_freq=min_freq, max_freq=max_freq)
frequencies = get_frequencies(FS)[bins]
print(frequencies)

# which sources to play at which angle
degree_list = [0, 27, 54, 81, 360]
sources = {
    deg: 'mono4125' if deg == 360 else 'sweep' for deg in degree_list 
}
distance_list = np.arange(100, step=10)

idx_all = 0
with progressbar.ProgressBar(max_value=len(distance_list) * len(degree_list)) as bar:
    for distance_cm in distance_list:
        for degree in degree_list:
            if degree == 360:
                signals_f = np.empty((0, 4, len(frequencies)))

                # we want to be done with a turn of 360 degrees in ca. 90 steps.
                degrees = np.linspace(0, 360, 90)
                for deg in degrees:
                    room = generate_room(distance_cm=distance_cm, yaw_deg=deg, source=sources[degree])
                    sig_f = get_signals_f(room, n_times=1)[:, :, bins]
                    signals_f = np.concatenate([sig_f, signals_f], axis=0)

                # fill the leftover buffer with measurements at 360
                sig_f = get_signals_f(room, n_times=N_TIMES)[:, :, bins]
                n_buffer_wait = int(FS * (DURATION_SEC - (signals_f.shape[0] * N_BUFFER * 2 / FS)) // (N_BUFFER * 2))
                signals_f = np.concatenate([sig_f[:n_buffer_wait, ...], signals_f])

                yaw = np.full(signals_f.shape[0], -360)
                yaw[:len(degrees)] = -degrees
            else:
                room = generate_room(distance_cm=distance_cm, yaw_deg=degree, source=sources[degree])
                signals_f = get_signals_f(room)[:, :, bins]
                yaw = np.full(signals_f.shape[0], -degree)

            spec = np.sum(np.abs(signals_f), axis=1)
            psd = get_psd(signals_f, frequencies, fname='simulated')

            df_total.loc[len(df_total), :] = dict(
                degree=degree,
                yaw=yaw,
                distance=distance_cm,
                source=sources[degree],
                signals_f=signals_f,
                frequencies=frequencies,
                spec=spec,
                psd=psd
            ) 
            idx_all += 1
            bar.update(idx_all)
        
#fname = f'results/{exp_name}_simulated.pkl'
#pd.to_pickle(df_total, fname)
#print('saved as', fname)